## Principal Component Analysis (PCA)

In this notebook, we will demonstrate the end-to-end workflow of Spark RAPIDS accelerated PCA.

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf

In [3]:
import os
import requests

SPARK_RAPIDS_VERSION = "24.08.1"
rapids_jar = f"rapids-4-spark_2.12-{SPARK_RAPIDS_VERSION}.jar"

if not os.path.exists(rapids_jar):
    print("Downloading spark rapids jar")
    url = f"https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/{SPARK_RAPIDS_VERSION}/{rapids_jar}"
    response = requests.get(url)
    if response.status_code == 200:
        with open(rapids_jar, "wb") as f:
            f.write(response.content)
        print(f"File '{rapids_jar}' downloaded and saved successfully.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print("File already exists. Skipping download.")

_config = {
    "spark.master": f"local[8]",
    "spark.driver.host": "127.0.0.1",
    "spark.task.maxFailures": "1",
    "spark.driver.memory": "10g",
    "spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled": "false",
    "spark.sql.pyspark.jvmStacktrace.enabled": "true",
    "spark.sql.execution.arrow.pyspark.enabled": "true",
    "spark.rapids.ml.uvm.enabled": "true",
    "spark.jars": rapids_jar,
    "spark.executorEnv.PYTHONPATH": rapids_jar,
    "spark.rapids.memory.gpu.minAllocFraction": "0.0001",
    "spark.plugins": "com.nvidia.spark.SQLPlugin",
    "spark.locality.wait": "0s",
    "spark.sql.cache.serializer": "com.nvidia.spark.ParquetCachedBatchSerializer",
    "spark.rapids.memory.gpu.pooling.enabled": "false",
    "spark.rapids.sql.explain": "ALL",
    "spark.sql.execution.sortBeforeRepartition": "false",
    "spark.rapids.sql.format.parquet.reader.type": "MULTITHREADED",
    "spark.rapids.sql.format.parquet.multiThreadedRead.maxNumFilesParallel": "20",
    "spark.rapids.sql.multiThreadedRead.numThreads": "20",
    "spark.rapids.sql.python.gpu.enabled": "true",
    "spark.rapids.memory.pinnedPool.size": "2G",
    "spark.python.daemon.module": "rapids.daemon",
    "spark.rapids.sql.batchSizeBytes": "512m",
    "spark.sql.adaptive.enabled": "false",
    "spark.sql.files.maxPartitionBytes": "512m",
    "spark.rapids.sql.concurrentGpuTasks": "1",
    "spark.sql.execution.arrow.maxRecordsPerBatch": "20000",
    "spark.rapids.sql.explain": "NONE",
}
spark = SparkSession.builder.appName("spark-rapids-ml")
for key, value in _config.items():
    spark = spark.config(key, value)
spark = spark.getOrCreate()

File already exists. Skipping download.


24/09/30 20:01:38 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
24/09/30 20:01:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/30 20:01:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/30 20:01:39 WARN RapidsPluginUtils: RAPIDS Accelerator 24.08.1 using cudf 24.08.0, private revision 9fac64da220ddd6bf5626bd7bd1dd74c08603eac
24/09/30 20:01:39 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
24/09/30 20:01:42 WARN GpuDeviceManager: RMM pool is disabled since spark.rapids.memory.gpu.pooling.enabled is set to false; however, this configuration is deprecated and the behavior may change in a futur

### Generate synthetic dataset

Here we generate a 100,000 x 2048 random dataset.

In [4]:
rows = 100000
dim = 2048
dtype = 'float32'
np.random.seed(42)

data = np.random.rand(rows, dim).astype(dtype)
pd_data = pd.DataFrame({"features": list(data)})
prepare_df = spark.createDataFrame(pd_data).cache()
prepare_df.write.mode("overwrite").parquet("PCA_data.parquet")

24/09/30 20:01:48 WARN TaskSetManager: Stage 0 contains a task of very large size (160085 KiB). The maximum recommended task size is 1000 KiB.


In [5]:
df = spark.read.parquet("PCA_data.parquet")
df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



### ETL: Mean-centering

PCA expects mean-centered data as input so that the first principal component is not influenced by the distribution mean. We perform a simple mean centering on the data below.

In [6]:
avg_values = df.select([
    F.avg(F.col("features")[i]).alias(f"avg_{i}") for i in range(dim)
]).first()

@pandas_udf(ArrayType(FloatType()))
def mean_center_udf(features: pd.Series) -> pd.Series:
    return features.apply(lambda row: [row[i] - avg_values[i] for i in range(dim)])

mean_centered_df = df.withColumn("mean_centered_features", mean_center_udf(F.col("features")))

24/09/30 20:01:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/09/30 20:01:58 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/09/30 20:02:05 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


#### Spark-RAPIDS-ML accepts ArrayType input

Note that in the original Spark-ML PCA, we must `Vectorize` the input column:

```python
from pyspark.ml.linalg import Vectors
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
    (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
    (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data,["features"])
df.show()
```

...whereas the Spark-RAPIDS-ML version does not require extra Vectorization, and can accept an ArrayType column as the input column:

In [7]:
data_df = mean_centered_df.withColumn("features", F.col("mean_centered_features")).drop("mean_centered_features")
data_df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



### Using Spark-RAPIDS-ML PCA (GPU)

Compared to the Spark-ML PCA training API:

```python
from pyspark.ml.feature import PCA
pca = PCA(k=3, inputCol="features")
pca.setOutputCol("pca_features")
```

We use a customized class which requires **no code change** from the user to enjoy GPU acceleration:

```python
from spark_rapids_ml.feature import PCA
pca = PCA(k=3, inputCol="features")
pca.setOutputCol("pca_features")
```

In [8]:
from spark_rapids_ml.feature import PCA

gpu_pca = PCA(k=2, inputCol="features")
gpu_pca.setOutputCol("pca_features")

PCA_a412248e0bfc

The PCA estimator object can be persisted and reloaded.

In [9]:
estimator_path = "/tmp/pca_estimator"
gpu_pca.write().overwrite().save(estimator_path)
gpu_pca_loaded = PCA.load(estimator_path)

#### Fit

In [10]:
start_time = time.time()
gpu_pca_model = gpu_pca_loaded.fit(data_df)
gpu_fit_time = time.time() - start_time
print(f"GPU PCA fit took: {gpu_fit_time} sec")

INFO: Process 3433163 found CUDA visible device(s): 0
2024-09-30 20:02:08,260 - spark_rapids_ml.feature.PCA - INFO - CUDA managed memory enabled.
2024-09-30 20:02:08,296 - spark_rapids_ml.feature.PCA - INFO - Training spark-rapids-ml with 1 worker(s) ...
2024-09-30 20:02:36,241 - spark_rapids_ml.feature.PCA - INFO - Loading data into python worker memory
2024-09-30 20:02:37,199 - spark_rapids_ml.feature.PCA - INFO - Initializing cuml context
2024-09-30 20:02:38,418 - spark_rapids_ml.feature.PCA - INFO - Invoking cuml fit
2024-09-30 20:02:39,938 - spark_rapids_ml.feature.PCA - INFO - Cuml fit complete
2024-09-30 20:02:41,478 - spark_rapids_ml.feature.PCA - INFO - Finished training


GPU PCA fit took: 34.151185512542725 sec


#### Transform

In [12]:
start_time = time.time()
embeddings = gpu_pca_model.transform(data_df).select("pca_features").show(truncate=False)
gpu_transform_time = time.time() - start_time
print(f"GPU PCA transform took: {gpu_transform_time} sec")

+----------------------------+
|pca_features                |
+----------------------------+
|[-0.029186783, -0.14947692] |
|[-0.113819614, -0.3001569]  |
|[0.24490958, 0.38476408]    |
|[0.40183866, -0.075875156]  |
|[0.3386735, 0.33758926]     |
|[-0.42260733, -0.052879076] |
|[0.31347108, 0.1891313]     |
|[0.48139828, 0.1312784]     |
|[0.2473749, -0.6211766]     |
|[-0.70062363, -0.42159277]  |
|[-0.34105372, -0.1214372]   |
|[0.05086224, 0.14144382]    |
|[0.22230142, 0.2221222]     |
|[0.25632417, 0.035815906]   |
|[0.6046144, 0.44387817]     |
|[-0.25216517, 0.1838089]    |
|[-0.22000793, 0.4833736]    |
|[-0.28257906, -0.0045994027]|
|[-0.3213926, 0.52787125]    |
|[0.102127835, 0.078904025]  |
+----------------------------+
only showing top 20 rows

GPU PCA transform took: 0.44330501556396484 sec


### Using Spark-ML PCA (CPU)

In [13]:
from pyspark.ml.feature import PCA

cpu_pca = PCA(k=2, inputCol="features")
cpu_pca.setOutputCol("pca_features")

PCA_89fd00984029

In [14]:
from pyspark.ml.functions import array_to_vector

vector_df = data_df.select(array_to_vector("features").alias("features"))

vector_df.printSchema()
vector_df.show(5, False)

root
 |-- features: vector (nullable = true)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Fit

In [15]:
start_time = time.time()
cpu_pca_model = cpu_pca.fit(vector_df)
pca_fit_time = time.time() - start_time
print(f"CPU PCA fit took: {pca_fit_time} sec")

INFO: Process 3433953 found CUDA visible device(s): 0                           
INFO: Process 3434065 found CUDA visible device(s): 0                           
INFO: Process 3434069 found CUDA visible device(s): 0
INFO: Process 3434073 found CUDA visible device(s): 0
INFO: Process 3434077 found CUDA visible device(s): 0
INFO: Process 3434081 found CUDA visible device(s): 0
INFO: Process 3434087 found CUDA visible device(s): 0
INFO: Process 3434092 found CUDA visible device(s): 0
INFO: Process 3434098 found CUDA visible device(s): 0
INFO: Process 3434702 found CUDA visible device(s): 0                           
24/09/30 20:05:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


CPU PCA fit took: 170.7325849533081 sec


#### Transform

In [16]:
start_time = time.time()
embeddings = cpu_pca_model.transform(vector_df).select("pca_features").show(truncate=False)
pca_transform_time = time.time() - start_time
print(f"CPU PCA transform took: {pca_transform_time} sec")

INFO: Process 3436216 found CUDA visible device(s): 0


+-------------------------------------------+
|pca_features                               |
+-------------------------------------------+
|[-0.02891399091249798,-0.14877800281964723]|
|[-0.11377219195105344,-0.3028603508804788] |
|[0.24481776731137372,0.3833952120253239]   |
|[0.40126459354751076,-0.07748546383735228] |
|[0.3382728256029929,0.33890683025531254]   |
|[-0.4228363394059959,-0.054901379310494144]|
|[0.31333870346431664,0.18913735472304322]  |
|[0.4810119397199453,0.13250040506577465]   |
|[0.24748029515379566,-0.6211263064522627]  |
|[-0.6999917660681605,-0.4207849872769174]  |
|[-0.3404634760494078,-0.11978179300560246] |
|[0.050740398452547276,0.1368302804571385]  |
|[0.22282065118767827,0.22023244883069099]  |
|[0.2562068262435786,0.03528786064786128]   |
|[0.6045398358884383,0.44301892614622806]   |
|[-0.2520494600322001,0.18266864414559097]  |
|[-0.2200009600414218,0.48386979207780034]  |
|[-0.2822597329505147,-0.006133424943223282]|
|[-0.321516215763547,0.52631782545

### Summary

With our 100,000 x 2048 dataset, we achieved end-to-end speedup of  

CPU: (170.73s + 0.50s)  
GPU: (34.15s + 0.44s)  

`CPU / GPU = 4.95`